# Mouse metagenomics for Parkinson Disease (Tutorial)

# 1. Metadata handling

In [ ]:
# Get the metadata file and store it into data/
!wget -nv -O 'data/metadata.tsv' \
	https://data.qiime2.org/2024.10/tutorials/pd-mice/sample_metadata.tsv

2024-11-14 02:49:51 URL:https://doc-00-60-sheets.googleusercontent.com/pub/54bogvaave6cua4cdnls17ksc4/kfkmgnll7qd9i6vvdfmmmr3ph4/1731570590000/105250506097979753968/*/e@2PACX-1vQ3g65vwLGsdtnARKVjuJlJ9wVPOUocJvoluuaB2PO8zA2Q2AwG9q84UvdUtcKkFGVK_32gcFKyqxbE?gid=1509704122&single=true&output=tsv [4431] -> "data/metadata.tsv" [1]


In [7]:
# Summarize and explore the metadata.
!qiime metadata tabulate \
  --m-input-file metadata.tsv \
  --o-visualization metadata.qzv

Saved Visualization to: metadata.qzv


In [ ]:
# Visualize the output
!qiime tools view metadata.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

# 2. Importing data into QIIME 2¶


In [ ]:
# Get the manifest file and store it into data/
!wget -nv -O 'manifest.tsv' \
	https://data.qiime2.org/2024.10/tutorials/pd-mice/manifest

2024-11-14 03:25:17 URL:https://s3-us-west-2.amazonaws.com/qiime2-data/2024.10/tutorials/pd-mice/manifest [4640/4640] -> "data/manifest.tsv" [1]


In [11]:
# Download sequences
!wget -nv -O "demultiplexed_seqs.zip" \
  "https://data.qiime2.org/2024.10/tutorials/pd-mice/demultiplexed_seqs.zip"

2024-11-14 03:29:45 URL:https://s3-us-west-2.amazonaws.com/qiime2-data/2024.10/tutorials/pd-mice/demultiplexed_seqs.zip [21508775/21508775] -> "demultiplexed_seqs.zip" [1]


In [12]:
# unzip and delete
!unzip demultiplexed_seqs.zip
!rm demultiplexed_seqs.zip

Archive:  demultiplexed_seqs.zip
   creating: demultiplexed_seqs/
  inflating: demultiplexed_seqs/10483.recip.539.ASO.PD4.D7_4_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.539.ASO.PD4.D14_5_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.413.WT.HC2.D7_12_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.220.WT.OB1.D7_30_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.458.ASO.HC3.D49_2_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.538.WT.PD4.D21_4_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.459.WT.HC3.D14_2_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.461.ASO.HC3.D7_20_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.465.ASO.PD3.D14_16_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.461.ASO.HC3.D21_11_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.540.ASO.HC4.D7_7_L001_R1_001.fastq.gz  
  i

In [8]:
# import the data
!qiime tools import \
  --type "SampleData[SequencesWithQuality]" \
  --input-format SingleEndFastqManifestPhred33V2 \
  --input-path ./manifest.tsv \
  --output-path ./demux_seqs.qza

Imported ./manifest.tsv as SingleEndFastqManifestPhred33V2 to ./demux_seqs.qza


In [9]:
# Output visualization
!qiime demux summarize \
  --i-data ./demux_seqs.qza \
  --o-visualization ./demux_seqs.qzv

mkdir: cannot create directory ‘output’: File exists
Saved Visualization to: ./demux_seqs.qzv


In [ ]:
# Visualize results
!qiime tools view ./demux_seqs.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

# 3. Sequence quality control (denoising) and feature table

In [10]:
# Denoise the data
!qiime dada2 denoise-single \
  --i-demultiplexed-seqs ./demux_seqs.qza \
  --p-trunc-len 150 \
  --o-table ./dada2_table.qza \
  --o-representative-sequences ./dada2_rep_set.qza \
  --o-denoising-stats ./dada2_stats.qza

Saved FeatureTable[Frequency] to: ./dada2_table.qza
Saved FeatureData[Sequence] to: ./dada2_rep_set.qza
Saved SampleData[DADA2Stats] to: ./dada2_stats.qza


In [11]:
# denoise the data statistically
!qiime metadata tabulate \
  --m-input-file ./dada2_stats.qza  \
  --o-visualization ./dada2_stats.qzv

Saved Visualization to: ./dada2_stats.qzv


In [ ]:
# Visualize table
!qiime tools view ./dada2_stats.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [12]:
# Feature table summary
!qiime feature-table summarize \
  --i-table ./dada2_table.qza \
  --m-sample-metadata-file ./metadata.tsv \
  --o-visualization ./dada2_table.qzv

Saved Visualization to: ./dada2_table.qzv


# 4. Generating a phylogenetic tree for diversity analysis

In [15]:
# Fragment insertion plugin suggest that it can outperform traditional alignment based methods based on short Illumina reads by alignment against a reference tree built out of larger sequences
!wget -nv -O "./sepp-refs-gg-13-8.qza" \
  "https://data.qiime2.org/classifiers/sepp-ref-dbs/sepp-refs-gg-13-8.qza"

2024-11-18 08:24:37 URL:https://s3-us-west-2.amazonaws.com/qiime2-data/classifiers/sepp-ref-dbs/sepp-refs-gg-13-8.qza [50161069/50161069] -> "./sepp-refs-gg-13-8.qza" [1]


In [16]:
# create a fragment insertion tree by fragment insertion
!qiime fragment-insertion sepp \
  --i-representative-sequences ./dada2_rep_set.qza \
  --i-reference-database ./sepp-refs-gg-13-8.qza \
  --o-tree ./tree.qza \
  --o-placements ./tree_placements.qza \
  --p-threads 2  # update to a higher number if you can

# explore outcome in iTOL

Saved Phylogeny[Rooted] to: ./tree.qza
Saved Placements to: ./tree_placements.qza


# 5. Alpha Rarefaction and Selecting a Rarefaction Depth

In [3]:
# calculate rarefaction
!mkdir diversity
!qiime diversity alpha-rarefaction \
  --i-table ./denoised/dada2_table.qza \
  --m-metadata-file ./data/metadata.tsv \
  --o-visualization ./diversity/alpha_rarefaction_curves.qzv \
  --p-min-depth 10 \
  --p-max-depth 4250

mkdir: cannot create directory ‘diversity’: File exists
Saved Visualization to: ./diversity/alpha_rarefaction_curves.qzv


In [ ]:
# Visualize table
!qiime tools view ./alpha_rarefaction_curves.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

# 6. Diversity analysis


In [20]:
# Generate core metric phylogenetics using the boundaries provided by rarefaction
!qiime diversity core-metrics-phylogenetic \
	--i-phylogeny tree.qza \
  --i-table dada2_table.qza \
	--p-sampling-depth 2000 \
  --m-metadata-file metadata.tsv \
  --output-dir core-metrics-results

Plugin error from diversity:

  module 'skbio.diversity.alpha' has no attribute 'sobs'

Debug info has been saved to /tmp/qiime2-q2cli-err-ya0n6ne1.log


In [3]:
!cat /tmp/qiime2-q2cli-err-bajpmiyk.log

Traceback (most recent call last):
  File "/home/davo/miniconda3/envs/qiime2-amplicon/lib/python3.10/site-packages/q2cli/commands.py", line 530, in __call__
    results = self._execute_action(
  File "/home/davo/miniconda3/envs/qiime2-amplicon/lib/python3.10/site-packages/q2cli/commands.py", line 608, in _execute_action
    results = action(**arguments)
  File "<decorator-gen-201>", line 2, in core_metrics_phylogenetic
  File "/home/davo/miniconda3/envs/qiime2-amplicon/lib/python3.10/site-packages/qiime2/sdk/action.py", line 299, in bound_callable
    outputs = self._callable_executor_(
  File "/home/davo/miniconda3/envs/qiime2-amplicon/lib/python3.10/site-packages/qiime2/sdk/action.py", line 651, in _callable_executor_
    outputs = self._callable(ctx, **view_args)
  File "/home/davo/miniconda3/envs/qiime2-amplicon/lib/python3.10/site-packages/q2_diversity/_core_metrics.py", line 60, in core_metrics_phylogenetic
    cr = core_metrics(table=table, sampling_depth=sampling_depth,
  File 

# 7. Taxonomic classification


In [ ]:
# Download a pre-trained naive Bayes machine-learning classifier
!mkdir taxonomic_profile
!wget -nv -O "./taxonomic_profile/gg-13-8-99-515-806-nb-classifier.qza" \
  "https://data.qiime2.org/classifiers/sklearn-1.4.2/greengenes/gg-13-8-99-515-806-nb-classifier.qza"

# Naive Bayes classifiers perform best when they’re trained for the specific hypervariable region amplified.

2024-11-14 05:49:57 URL:https://s3-us-west-2.amazonaws.com/qiime2-data/classifiers/sklearn-1.4.2/greengenes/gg-13-8-99-515-806-nb-classifier.qza [2334033/2334033] -> "./taxonomic_profile/gg-13-8-99-515-806-nb-classifier.qza" [1]


In [34]:
#  train a classifier specific for your dataset
!qiime feature-classifier classify-sklearn \
  --i-reads ./denoised/dada2_rep_set.qza \
  --i-classifier ./taxonomic_profile/gg-13-8-99-515-806-nb-classifier.qza \
  --o-classification ./taxonomic_profile/taxonomy.qza

Saved FeatureData[Taxonomy] to: ./taxonomic_profile/taxonomy.qza


In [35]:
# Review the taxonomy associated with the sequences
!qiime metadata tabulate \
  --m-input-file ./taxonomic_profile/taxonomy.qza \
  --o-visualization ./taxonomic_profile/taxonomy.qzv

Saved Visualization to: ./taxonomic_profile/taxonomy.qzv


In [36]:
# Visualize table
!qiime tools view ./taxonomic_profile/taxonomy.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [37]:
# Visualize feature table
!qiime feature-table tabulate-seqs \
  --i-data ./denoised/dada2_rep_set.qza \
  --o-visualization ./taxonomic_profile/dada2_rep_set.qzv

Saved Visualization to: ./taxonomic_profile/dada2_rep_set.qzv


In [39]:
# Visualize table
!qiime tools view ./taxonomic_profile/dada2_rep_set.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

## Taxonomy Barchart

In [42]:
# filter out samples with fewer sequences than our rarefaction depth
!qiime feature-table filter-samples \
  --i-table ./denoised/dada2_table.qza \
  --p-min-frequency 2000 \
  --o-filtered-table ./taxonomic_profile/table_2k.qza

Saved FeatureTable[Frequency] to: ./taxonomic_profile/table_2k.qza


In [43]:
# Build interactive barplot
!qiime taxa barplot \
  --i-table ./taxonomic_profile/table_2k.qza \
  --i-taxonomy ./taxonomic_profile/taxonomy.qza \
  --m-metadata-file ./data/metadata.tsv \
  --o-visualization ./taxonomic_profile/taxa_barplot.qzv

Saved Visualization to: ./taxonomic_profile/taxa_barplot.qzv


In [44]:
# Visualize barplot
!qiime tools view ./taxonomic_profile/taxa_barplot.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.Opening in existing browser session.

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.